In [17]:
# class에 대해서 조금 만 더 알아보자 
# class는 객체를 만들기 위해서 필요한 틀 
# class안에 기술된 정보를 바탕으로 일정 메모리 영역을 확봘 수 있고 이 영역이 객체

class Car:
    # constructor(생성자) -> 함수 
    # 객체가 처음 만들어질 때 무조건 호출된다. 
    # 객체가 가지는 변수를 특정 값으로 초기화 
    def __init__(self,name):
        print("객체가 생성되요")
        # class로무터 파생된 객체가 가질 수 있는 data 는 
        # 여기에 명시한다. 
        #self.car_name = "소나타" 
        self.car_name = name
        self.fuel = 100 #연관성을 갖는 객체 하나 선언 
                        #객체내 변수와 기능은 연관성이 있다. 
                        #따로따로 노는 것이 아닌 기능은 변수를 제어하는 용도 
                        #(꼭 그런것은 아니지만 거의 예외없이)

    # 기능을 위한 함수(여러개 나올 수 있다. )
    def go_front(self):
        my_car = 100 # self가 있어 함수 안에 국한되는 변수로 인식하지않는다
                     # 객체가 갖는 변수로 인식  
        print("자동차가 전진해요!") 
        self.fuel = self.fuel - 5 #self를 안쓴다면 함수내부로 국한됨 
        #fuel = fuel - 5          #애초에 이름도 다름 (self.fuel != fuel) 

    def backward(self):
        print("자동차가 후진해요!")
        self.fuel = self.fuel - 3  

    def get_fuel(self): #잔여 연료량을 알려주는 함수 #self꼭 기재 
        return self.fuel 




car1 = Car("소나타") #car1이 하나의 객체 
car1.go_front()
car1.backward()
print(car1.car_name)
#car1.car_name = "그랜저"
car2 = Car("그랜저") #car2는 하나의 객체 
car2.backward() #"." -> dot operator(점 연산자): '이 객체가 가지고 있는 기능'으로 쓰임 
print(car2.car_name)
print(car1.get_fuel())

객체가 생성되요
자동차가 전진해요!
자동차가 후진해요!
소나타
객체가 생성되요
자동차가 후진해요!
그랜저
92


In [29]:
### CNN(ensemble)을 class를 이용해서 구현해 보아요! 
## ensemble은 여러개의 CNN model을 만들어서 이용 
## 객체를 만들어서 각 개체에 CNN model을 하나씩 저장 

import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data 
import pandas as pd 
import numpy as np 

# 그래프 초기화 
tf.reset_default_graph()

class CnnModel:
    def __init__(self,session, data): #class로 만들시 session을 init 파트에서 실시한다
        print("객체가 생성되었어요!")
        self.build_graph()
        self.sess = session
        self.mnist = data 

    def build_graph(self):
        print("텐서플로우 그래프를 그려요!")
        # 직접쓰는 코드와 내부에서만 쓰는 코드 구분하여 self를 붙이자 
        # 2.1 placeholder
        self.X = tf.placeholder(shape=[None,784], dtype=tf.float32)
        self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
        #※ tensorflow 버전에 따라서 dropout을 사용하는 속성이 변경 
        #  최신버전(버리는쪽) -> rate =0.8 : drop 시키는 노드의 비율 
        #  이전버전(살리는쪽) -> keep_prob = 0.8 : 살리는 노드의 비율  
        self.keep_rate = tf.placeholder(dtype=tf.float32)

        # 2.2 Convolution Layer 
        # 입력데이터는 4차원 
        x_img = tf.reshape(self.X,[-1,28,28,1])
        L1 = tf.layers.conv2d(inputs=x_img, filters=32,
                              kernel_size = [3,3], 
                              padding = "SAME", #기존 적었던 내용들을 함수의 인자들로
                              strides = 1, 
                              activation =tf.nn.relu)   
        L1 = tf.layers.max_pooling2d(inputs = L1, 
                                    pool_size=[2,2],
                                    padding = "SAME", 
                                    strides=2)
        L1 = tf.layers.dropout(inputs = L1, rate= 1- self.keep_rate)



        L2 = tf.layers.conv2d(inputs=L1, filters=64,
                            kernel_size = [3,3], padding = "SAME", #기존 적었던 내용들을 함수의 인자들로
                            strides = 1, activation =tf.nn.relu)   
        L2= tf.layers.max_pooling2d(inputs = L2, pool_size=[2,2],
                                    padding = "SAME", strides=2)
        L2= tf.layers.dropout(inputs = L2, rate= 1 - self.keep_rate)



        # 2.3 FC layer (=dense layer)
        # dense 함수 설정하면 세줄짜리 코드를 한줄분량으로 설정 가능 
        L2 = tf.reshape(L2,[-1,7*7*64])
        dense1 = tf.layers.dense(inputs = L2, units=256, 
                         activation = tf.nn.relu)
        dense1 = tf.layers.dropout(inputs=dense1, rate= 1 - self.keep_rate)


        dense2= tf.layers.dense(inputs = dense1, units=128, 
                         activation = tf.nn.relu)
        dense2= tf.layers.dropout(inputs=dense2, rate= 1 - self.keep_rate)


        dense3 = tf.layers.dense(inputs = dense2, units=512, 
                         activation = tf.nn.relu)
        dense3 = tf.layers.dropout(inputs=dense3, rate= 1 - self.keep_rate)


        #Hypothesis 
        self.H = tf.layers.dense(inputs = dense3, units=10)

                    
        #Cost
        self.cost = tf.losses.softmax_cross_entropy(self.Y,self.H)

        #train
        self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)
    
    def train_graph(self):
        print("텐서플로우 그래프를 학습 시켜요") 
        #학습
        self.sess.run(tf.global_variables_initializer())
        num_of_epoch = 30 
        batch_size = 100 

        for step in range(num_of_epoch):
            num_of_iter = int(mnist.train.num_examples / batch_size)
            cost_val = 0 # 코스트 초기화(국룰)
            
            for i in range(num_of_iter):
                batch_x,batch_y = mnist.train.next_batch(batch_size)
                _, cost_val = self.sess.run([self.train,self.cost], 
                                            feed_dict={self.X:batch_x, 
                                                        self.Y:batch_y,
                                                        self.keep_rate:0.7 })
            if step % 3 == 0 :
                print("Cost는 : {}".format(cost_val))

    def get_accuracy(self):
        print("정확도를 구해요!")
        predict = tf.argmax(self.H,1)
        correct = tf.equal(predict, tf.argmax(self.Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

        print("정확도는 : {}".format(sess.run(accuracy, 
                                        feed_dict={self.X:mnist.test.images, 
                                                   self.Y:mnist.test.labels,
                                                   self.keep_rate:1.0})))
    '''
    def get_accuracy(self, x_test, y_test, keep_prop=1.0):

        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})
    '''
    '''
    def get_prediction(self):
        print("예측값을 구해요!")
        return self.sess.run(self.logits,
                             feed_dict = {self.X: 
                                          self.Y:
                                          self.keep_rate: 1.0})
    '''
# 모델 10개를 만들꺼예요! : 객체 10개를 만들거예요 
'''
model1= CnnModel()
model2 = CnnModel()
model3 = CnnModel()
model4 = CnnModel()
'''

# Ensamble
sess = tf.Session()
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)
num_of_model = 10
models = [CnnModel(sess, mnist) for x in range(num_of_model)] #CnnModel()를 호출해서 list안에 하나씩 집어넣는다 

for i in range(num_of_model):
    models[i].train_graph()
    models[i].get_accuracy()

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
텐서플로우 그래프를 학습 시켜요
Cost는 : 0.02375377155840397
Cost는 : 0.01411296334117651
Cost는 : 0.005204381421208382
Cost는 : 0.00036485790042206645
Cost는 : 0.045848846435546875
Cost는 : 0.0041394843719899654
Cost는 : 8.219357550842687e-05
Cost는 : 1.89319525816245e-05
Cost는 : 6.355872756103054e-05
Cost는 : 0.00013122310338076204
정확도를 구해요!
정확도는 : 0.9915000200271606
텐서플로우 그래프를 학습 시켜요
Cost는 : 0.0443902388215065
Cost는 : 0.022927463054656982
Cost는 : 0.00038397058960981667
Cost는 : 0.002552641090005636
Cost는 : 0.0009932605316862464
Cost는

In [0]:
class Model:



    def __init__(self, sess, name):

        self.sess = sess

        self.name = name

        self._build_net()



    def _build_net(self):

        with tf.variable_scope(self.name):

            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1

            # for testing

            self.keep_prob = tf.placeholder(tf.float32)



            # input place holders

            self.X = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white)

            X_img = tf.reshape(self.X, [-1, 28, 28, 1])

            self.Y = tf.placeholder(tf.float32, [None, 10])



            # L1 ImgIn shape=(?, 28, 28, 1)

            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))

            #    Conv     -> (?, 28, 28, 32)

            #    Pool     -> (?, 14, 14, 32)

            L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')

            L1 = tf.nn.relu(L1)

            L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],

                                strides=[1, 2, 2, 1], padding='SAME')

            L1 = tf.nn.dropout(L1, keep_prob=self.keep_prob)

            '''

            Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)

            Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)

            Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)

            Tensor("dropout/mul:0", shape=(?, 14, 14, 32), dtype=float32)

            '''



            # L2 ImgIn shape=(?, 14, 14, 32)

            W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))

            #    Conv      ->(?, 14, 14, 64)

            #    Pool      ->(?, 7, 7, 64)

            L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')

            L2 = tf.nn.relu(L2)

            L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],

                                strides=[1, 2, 2, 1], padding='SAME')

            L2 = tf.nn.dropout(L2, keep_prob=self.keep_prob)

            '''

            Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)

            Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)

            Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)

            Tensor("dropout_1/mul:0", shape=(?, 7, 7, 64), dtype=float32)

            '''



            # L3 ImgIn shape=(?, 7, 7, 64)

            W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))

            #    Conv      ->(?, 7, 7, 128)

            #    Pool      ->(?, 4, 4, 128)

            #    Reshape   ->(?, 4 * 4 * 128) # Flatten them for FC

            L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')

            L3 = tf.nn.relu(L3)

            L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[

                                1, 2, 2, 1], padding='SAME')

            L3 = tf.nn.dropout(L3, keep_prob=self.keep_prob)



            L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])

            '''

            Tensor("Conv2D_2:0", shape=(?, 7, 7, 128), dtype=float32)

            Tensor("Relu_2:0", shape=(?, 7, 7, 128), dtype=float32)

            Tensor("MaxPool_2:0", shape=(?, 4, 4, 128), dtype=float32)

            Tensor("dropout_2/mul:0", shape=(?, 4, 4, 128), dtype=float32)

            Tensor("Reshape_1:0", shape=(?, 2048), dtype=float32)

            '''



            # L4 FC 4x4x128 inputs -> 625 outputs

            W4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625],

                                 initializer=tf.contrib.layers.xavier_initializer())

            b4 = tf.Variable(tf.random_normal([625]))

            L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)

            L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)

            '''

            Tensor("Relu_3:0", shape=(?, 625), dtype=float32)

            Tensor("dropout_3/mul:0", shape=(?, 625), dtype=float32)

            '''



            # L5 Final FC 625 inputs -> 10 outputs

            W5 = tf.get_variable("W5", shape=[625, 10],

                                 initializer=tf.contrib.layers.xavier_initializer())

            b5 = tf.Variable(tf.random_normal([10]))

            self.logits = tf.matmul(L4, W5) + b5

            '''

            Tensor("add_1:0", shape=(?, 10), dtype=float32)

            '''



        # define cost/loss & optimizer

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(

            logits=self.logits, labels=self.Y))

        self.optimizer = tf.train.AdamOptimizer(

            learning_rate=learning_rate).minimize(self.cost)



        correct_prediction = tf.equal(

            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))

        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



    def predict(self, x_test, keep_prop=1.0):

        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.keep_prob: keep_prop})



    def get_accuracy(self, x_test, y_test, keep_prop=1.0):

        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})



    def train(self, x_data, y_data, keep_prop=0.7):

        return self.sess.run([self.cost, self.optimizer], feed_dict={

            self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})


